In [42]:
import numpy as np

In [43]:
data= open('dinos.txt','r').read()
data= data.lower()
unique_chars= list(set(data))

In [44]:
total_chars, total_unique_chars= len(data), len(unique_chars)

In [45]:
index_char= {}
char_index= {}
for i,v in enumerate(unique_chars):
    index_char[i]=v
    char_index[v]=i
    

In [46]:
train=[]
fo = open("dinos.txt", "r")
train = fo.readlines()
for i in range(0,len(train)):
    train[i]= train[i].lower().strip('\n')

In [86]:
epochs=2000
a_dim=100
x_dim=total_unique_chars
y_dim=total_unique_chars

Wax=np.random.uniform(-np.sqrt(1./a_dim), np.sqrt(1./a_dim), (a_dim, x_dim))
Wya=np.random.uniform(-np.sqrt(1./a_dim), np.sqrt(1./a_dim), (y_dim, a_dim))
Waa=np.random.uniform(-np.sqrt(1./a_dim), np.sqrt(1./a_dim), (a_dim, a_dim))
b= np.zeros((a_dim,1))
by= np.zeros((y_dim,1))
parameters={"Wax":Wax, "Wya":Wya, "Waa":Waa, "b":b, "by":by}

In [51]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [52]:
def sample(parameters, char_to_ix, seed):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    x = np.zeros((len(char_to_ix),1))
    a_prev = np.zeros((n_a, 1))
    indices = []
    idx = -1 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        a = np.tanh(Wax.dot(x) + Waa.dot(a_prev) + b)
        z = Wya.dot(a) + by
        y = softmax(z)
        np.random.seed(counter+seed) 
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
        indices.append(idx)
        x = np.zeros_like(x)
        x[idx] = 1
        a_prev = a
        seed += 1
        counter +=1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [53]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) 
    p_t = softmax(np.dot(Wya, a_next) + by) 
    return a_next, p_t

In [54]:
def rnn_forward(X, Y, a0, parameters):
    
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0
    
    for t in range(0,len(X)):
        x[t] = np.zeros((total_unique_chars,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

In [55]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


In [56]:
def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

In [57]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        gradient[:] = np.clip(gradient, -maxValue, maxValue)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [58]:
def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [59]:
def optimize(X,Y, a_prev, parameters, lr=0.01):
    loss, cache = rnn_forward(X,Y,a_prev,parameters)
    gradient ,a = rnn_backward(X,Y,parameters, cache)
    gradient    = clip(gradient,5)
    parameters  = update_parameters(parameters, gradient, lr)
        
    return loss, gradient, a[len(X)-1]

In [78]:
def print_sample(sample_ix):
    txt = ''.join(index_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:] 
    print ('%s' % (txt, ), end='')

In [87]:
def model(data):
    fo = open("dinos.txt", "r")
    train = fo.readlines()
    for i in range(0,len(train)):
        train[i]= train[i].lower().strip('\n')
    np.random.seed(0)
    np.random.shuffle(train)
    a_prev = np.zeros((a_dim, 1))
    for i in range(0,epochs):
        for j in range(0,len(train)):
            X = [None] + [char_index[ch] for ch in train[j]]
            Y = X[1:] + [char_index["\n"]]
            curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
            
        if(i%10==0):
            print('Epoch:%d Iteration:%d--->Loss:%f'%(i,j,curr_loss)+'\n')
            seed=0
            for name in range(0,5):
                sampled_indices = sample(parameters, char_index, seed)
                print_sample(sampled_indices)
                
                seed += 1 
            print('\n')
    return parameters

In [ ]:
parameters = model(data)

Epoch:0 Iteration:1535--->Loss:24.911477

Ashopeus
Qotyisaurus
Ambjeussaurcanocingasaurus
Aurisasaurus
Mchosaurusaurus


Epoch:10 Iteration:1535--->Loss:21.688634

Onwhonsaurus
Ans
Ambhosaurus
Otilunosaurus
Mechus


Epoch:20 Iteration:1535--->Loss:23.735607

Ashnpcoosaurus
Antuitnosuyrasous
Adbcpyosaurus
Allithos
Depkus


Epoch:30 Iteration:1535--->Loss:27.330989

Gydchortonoscunoclogagphiopamatlus
Rys
Rebpnuatoeshunoblosaurus
Aurisncaulupactyahatrpsaurusaurus
Fepbuagongwanoblosaurus


Epoch:40 Iteration:1535--->Loss:28.927819

Auhnphus
Auruiropalineapsaurus
Aochosaurus
Aurisaurus
Mophus


Epoch:50 Iteration:1535--->Loss:26.488801

Ersorous
Ossurunngulakaegusnasanvonnirnzngzyronbongongtosau
Ossurugunashunabingsaurus
Ersurungulakaegusznurhlynzuuhonsaurusyngtongtosaur
Mesourus


Epoch:60 Iteration:1535--->Loss:25.940408

Oraurus
Ors
Ossurus
Ors
Mesglongospanahingannolopshyalgusaurus


Epoch:70 Iteration:1535--->Loss:27.007590

Oreopotongospunopungannniaerhysiayszusaurus
Grslisaurus
Gesrp